In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/plant-seedlings-classification/train/Scentless Mayweed/4ae939d7d.png'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
# import pandas_profiling
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
pil_to_tensor = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5))])
tensor_to_pil = transforms.Compose([transforms.ToPILImage()])

In [ ]:
tmp_image = Image.open("/kaggle/input/plant-seedlings-classification/train/Scentless Mayweed/4ae939d7d.png")
scale_tmp_image = tmp_image.resize((200, 200))
numpy_tmp_image = np.array(scale_tmp_image)
plt.imshow(numpy_tmp_image)
plt.show()
numpy_tmp_image.shape

In [ ]:
c1 = nn.Conv2d(3, 3, kernel_size=2, stride=1)
c1_output = c1(pil_to_tensor(scale_tmp_image))

In [ ]:
print(c1_output.shape)
tensor_to_pil(c1_output)

In [ ]:
train_data_dir = '/kaggle/input/plant-seedlings-classification/train'
test_data_dir = '/kaggle/input/plant-seedlings-classification/test'

In [ ]:
def plot_image_path(img_path):
    img = Image.open(img_path)
    img_np_array = np.array(img)
    plt.imshow(img_np_array)

In [ ]:
class MapLabelToINT:
    def __init__(self, labels_list):
        self.labels_list = labels_list
        self.labels_map = {}
        for i in range(len(labels_list)):
            self.labels_map[labels_list[i]] = i
    
    def get_id(self, label):
        return self.labels_map[label]
    
    def get_class(self, class_id):
        for key, val in self.labels_map:
            if val==class_id:
                return key
        return None
    

    
label_names = os.listdir(train_data_dir)
print(label_names)

label_mapper = MapLabelToINT(label_names)

In [ ]:
# 70% data will be training data and 30% will be the test data
# here we will use this ratio for each class
validation_ratio = 0.3

train_data = []
validation_data = []

for class_name in label_names:
    class_path = os.path.join(train_data_dir, class_name)
    all_files =  [os.path.join(class_path, filename) for filename in os.listdir(class_path)]
    
    train, validation = train_test_split(all_files, shuffle=True, test_size=validation_ratio)
    
    for t_file in train:
        train_data.append([t_file, class_name])
    for v_file in validation:
        validation_data.append([v_file, class_name])


In [ ]:
print(len(train_data), len(validation_data))

In [ ]:
validation_data[0]

In [ ]:
class LabeledDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        img_path, c = self.data[index]
        img = Image.open(img_path).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
                
        return img, c       

In [ ]:
pil_data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

tensor_to_pil = transforms.Compose([
    transforms.ToPILImage()
])

In [ ]:
train_data_loader = LabeledDataset(train_data, pil_data_transforms)

img, c = train_data_loader.__getitem__(0)

pil_mg = tensor_to_pil(img)

In [ ]:
plt.imshow(pil_mg)

In [ ]:
plot_image_path(train_data[0][0])

In [ ]:
train_data_loader = LabeledDataset(train_data, pil_data_transforms)
validation_data_loader = LabeledDataset(validation_data, pil_data_transforms)


In [ ]:
train_loader = DataLoader(train_data_loader, shuffle=True, batch_size=8, num_workers=8) 
validation_loader = DataLoader(validation_data_loader, shuffle=True, batch_size=8, num_workers=8) 

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_0', pretrained=True)

In [ ]:
model

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

# https://pytorch.org/hub/pytorch_vision_squeezenet/

In [ ]:
def get_tensor_from_labels(labels, label_mapper):
    label_tensor = torch.zeros((len(labels), 12))
    
    for i in range(len(labels)):
        label_tensor[i][label_mapper.get_id(labels[i])] = 1.0
    return label_tensor



In [ ]:
torch.zeros((2,3))

In [ ]:
class NewModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.c1 = nn.Linear(1000, 12)
        self.c2 = nn.Softmax(dim=1)
    
    def forward(self, x):
        out1 = F.relu( self.c1(x))
        out2 = self.c2(out1)
        
        return out2

In [ ]:
epoches = 10


new_layer = NewModel()
model.train()
new_layer.train()
for params in model.parameters():
    params.requires_grad = False


for epoch in range(epoches):
    running_loss = 0.0
    for i, data in enumerate(iter(train_loader), 0):
        inputs, labels = data
        
        label_tensor = get_tensor_from_labels(labels, label_mapper)
        optimizer.zero_grad()
        
        _outputs = model(inputs)
        outputs = new_layer(_outputs)
        loss = loss_function(outputs, label_tensor)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 50 == 49:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0
    
    scheduler.step()

print('Finished Training')

In [ ]:
import pickle


def save_model(model, filename):
    base_path = "/kaggle/model/"
    os.makedirs(base_path, exist_ok=True)
    file_path = os.path.join(base_path, filename)
    pickle.dump(model, open(file_path, 'wb'))
    
    
    
def load_model(filename):
    base_path = "/kaggle/model/"
    file_path = os.path.join(base_path, filename)
    loaded_model = pickle.load(open(file_path, 'rb'))
    
    return loaded_model

In [ ]:
filename = "model_v2_linear_softmax.sav"

save_model(new_layer, filename)
# new_layer = load_model(filename)

In [ ]:
model.eval()
new_layer.eval()

In [ ]:
nn.Softmax()

In [ ]:
correct = 0
no_correct = 0
with torch.no_grad():
    for imgs, labels in validation_loader:
        _pred = model(imgs)
        output = new_layer(_pred)
        for i in range(len(labels)):
            pred = torch.argmax(output[i])
            if pred == label_mapper.get_id(labels[i]):
                correct += 1
            else:
                no_correct += 1
        

In [ ]:
dataiter = iter(validation_loader)
images, labels = dataiter.next()

In [ ]:
_out = model(images)
out = new_layer(_out)

print(out.shape)


In [ ]:
torch.argmax(out[0]), label_mapper.get_id(labels[0])

In [ ]:
print(correct, no_correct)

In [ ]:
out[0]

In [ ]:
outputs.shape

In [ ]:
labels

In [ ]:
label_mapper.get_id(labels[0])

In [ ]:
test_loader

In [ ]:
prediction = model(imgs)

In [ ]:
predictions[0].shape

In [ ]:
outputs